In [1]:
import warnings
import logging
import os
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

# Import Library

In [2]:
import os, sys, random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.amp import autocast, GradScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm

# Define Config & Prompt

In [3]:
# ====================CONFIG======================
import random, numpy as np, torch
import unicodedata, re

SEED = 42
BASE_MODEL     = "vinai/phobert-base"
MAX_LEN        = 256
BATCH_SIZE     = 64
NUM_EPOCHS     = 8
LABEL_SMOOTH   = 0.1
CLIP_GRAD_NORM = 1.0
WEIGHT_DECAY   = 3e-4
LR_BACKBONE    = 1e-5
LR_HEAD        = 1e-4
NEG_PER_SENT   = 2

ASPECTS = ["giang_vien", "chuong_trinh", "co_so_vat_chat", "khac"]
ASPECTS_VI = ["giang_vien", "chuong_trinh", "co_so_vat_chat", "khac"]
ASPECTS_EN = ["lecturer", "training_program", "facility", "others"]


def _norm_store(s: str) -> str:
    s = unicodedata.normalize("NFC", str(s)).strip()
    s = re.sub(r"\s+", " ", s)
    return s


def _norm_match(s: str) -> str:
    s = s.lower()
    s = unicodedata.normalize("NFD", s)
    return "".join(ch for ch in s if unicodedata.category(ch) != "Mn")


def _no_diacritics_set(kws: set) -> set:
    return kws | {_norm_match(k) for k in kws}

# ===== PROMPTS =====
ASPECT_PROMPTS = {
    "giang_vien": {
        "_default": (
            "ĐÁNH GIÁ phần liên quan GIẢNG VIÊN (giảng dạy, thái độ, hỗ trợ, chấm điểm, đúng giờ). Nếu câu không nhắc rõ đến GIẢNG VIÊN -> NONE. Mỗi aspect đánh giá độc lập (ví dụ: giảng viên đi dạy trễ nhưng mạng wifi tốt -> giảng viên NEGATIVE, cơ sở vật chất POSITIVE). NEGATIVE khi phàn nàn trễ, khó hiểu, thiếu hỗ trợ; POSITIVE khi được khen đúng giờ, nhiệt tình, dễ hiểu; không rõ -> NEUTRAL."
        ),
        "giang_day": (
            "ĐÁNH GIÁ GIẢNG DẠY của GIẢNG VIÊN. Nếu câu không nói về bài giảng, cách truyền đạt, phương pháp -> NONE. Mỗi aspect độc lập. NEGATIVE khi khó hiểu, quá nhanh/chậm, thiếu ví dụ; POSITIVE khi dễ hiểu, nhiều ví dụ, rõ ràng; không rõ -> NEUTRAL."
        ),
        "dung_gio": (
            "ĐÁNH GIÁ ĐÚNG GIỜ của GIẢNG VIÊN. Nếu câu không nhắc việc vào lớp, bắt đầu/kết thúc tiết -> NONE. Mỗi aspect độc lập. NEGATIVE khi trễ, bỏ tiết; POSITIVE khi đúng giờ, giữ lịch; không rõ -> NEUTRAL."
        ),
        "ho_tro": (
            "ĐÁNH GIÁ HỖ TRỢ/TƯ VẤN của GIẢNG VIÊN. Nếu câu không nhắc hỗ trợ, phản hồi, giải đáp -> NONE. Mỗi aspect độc lập. NEGATIVE khi chậm phản hồi, không giúp; POSITIVE khi nhiệt tình, phản hồi nhanh; không rõ -> NEUTRAL."
        ),
        "cham_diem": (
            "ĐÁNH GIÁ CHẤM ĐIỂM của GIẢNG VIÊN. Nếu câu không nói điểm, rubric, phúc khảo -> NONE. Mỗi aspect độc lập. NEGATIVE khi không công bằng, khó hiểu; POSITIVE khi minh bạch, công bằng; không rõ -> NEUTRAL."
        ),
        "thai_do": (
            "ĐÁNH GIÁ THÁI ĐỘ/TÁC PHONG của GIẢNG VIÊN. Nếu câu không nhắc thái độ, giao tiếp -> NONE. Mỗi aspect độc lập. NEGATIVE khi thô lỗ, thiếu tôn trọng; POSITIVE khi thân thiện, tôn trọng; không rõ -> NEUTRAL."
        ),
    },
    "chuong_trinh": {
        "_default": (
            "ĐÁNH GIÁ CHƯƠNG TRÌNH ĐÀO TẠO (môn học, tín chỉ, nội dung, lộ trình, lịch). Nếu câu không nhắc rõ đến chương trình -> NONE. Mỗi aspect đánh giá độc lập (ví dụ: lịch học dày nhưng giảng viên hỗ trợ tốt -> chương trình NEGATIVE, giảng viên POSITIVE). NEGATIVE khi quá tải, lạc hậu, trùng lặp; POSITIVE khi hợp lý, cập nhật, thực tế; không rõ -> NEUTRAL."
        ),
        "noi_dung": (
            "ĐÁNH GIÁ NỘI DUNG CHƯƠNG TRÌNH. Nếu câu không nói nội dung môn, học liệu, lộ trình -> NONE. Mỗi aspect độc lập. NEGATIVE khi lạc hậu, trùng lặp, thiếu thực tế; POSITIVE khi cập nhật, hữu ích; không rõ -> NEUTRAL."
        ),
        "lich_hoc": (
            "ĐÁNH GIÁ LỊCH HỌC/KẾ HOẠCH. Nếu câu không nhắc lịch, thời khóa biểu, xếp ca -> NONE. Mỗi aspect độc lập. NEGATIVE khi dồn dập, trùng lịch, đổi lịch liên tục; POSITIVE khi rõ ràng, hợp lý; không rõ -> NEUTRAL."
        ),
        "tin_chi": (
            "ĐÁNH GIÁ TÍN CHỈ/HỌC PHẦN. Nếu câu không nói tín chỉ, đăng ký học phần, tiên quyết -> NONE. Mỗi aspect độc lập. NEGATIVE khi bất hợp lý, khó đăng ký; POSITIVE khi phân bổ hợp lý, dễ đăng ký; không rõ -> NEUTRAL."
        ),
        "de_cuong": (
            "ĐÁNH GIÁ ĐỀ CƯƠNG/GIÁO TRÌNH. Nếu câu không nhắc đề cương, tài liệu, rubric -> NONE. Mỗi aspect độc lập. NEGATIVE khi thiếu rõ ràng, thiếu tài liệu; POSITIVE khi đầy đủ, minh bạch; không rõ -> NEUTRAL."
        ),
    },
    "co_so_vat_chat": {
        "_default": (
            "ĐÁNH GIÁ CƠ SỞ VẬT CHẤT (mạng, phòng học, phòng thí nghiệm, thiết bị, thư viện, gửi xe, vệ sinh, cổng đào tạo). Nếu câu không nhắc rõ đến cơ sở vật chất -> NONE. Mỗi aspect đánh giá độc lập (ví dụ: phòng học nóng nhưng thầy cô dạy dễ hiểu -> cơ sở vật chất NEGATIVE, giảng viên POSITIVE). NEGATIVE khi phàn nàn hỏng, thiếu, bẩn; POSITIVE khi khen đầy đủ, sạch, hiện đại; không rõ -> NEUTRAL."
        ),
        "mang": (
            "ĐÁNH GIÁ MẠNG/WI-FI. Nếu câu không nói mạng, wifi, internet -> NONE. Mỗi aspect độc lập. NEGATIVE khi chậm, rớt kết nối; POSITIVE khi nhanh, ổn định; không rõ -> NEUTRAL."
        ),
        "phong_hoc": (
            "ĐÁNH GIÁ PHÒNG HỌC. Nếu câu không nói phòng học, bàn ghế, điều hòa, tiếng ồn -> NONE. Mỗi aspect độc lập. NEGATIVE khi nóng, ồn, xuống cấp; POSITIVE khi mát, sạch, đủ tiện nghi; không rõ -> NEUTRAL."
        ),
        "phong_thi_nghiem": (
            "ĐÁNH GIÁ PHÒNG THÍ NGHIỆM/THỰC HÀNH. Nếu câu không nhắc lab, thiết bị thực hành -> NONE. Mỗi aspect độc lập. NEGATIVE khi thiếu máy, phần mềm lỗi; POSITIVE khi đầy đủ, hiện đại; không rõ -> NEUTRAL."
        ),
        "thiet_bi": (
            "ĐÁNH GIÁ THIẾT BỊ GIẢNG DẠY. Nếu câu không nói máy chiếu, micro, loa, TV -> NONE. Mỗi aspect độc lập. NEGATIVE khi hỏng, âm kém; POSITIVE khi hoạt động tốt, rõ ràng; không rõ -> NEUTRAL."
        ),
        "thu_vien": (
            "ĐÁNH GIÁ THƯ VIỆN. Nếu câu không nhắc thư viện, tài liệu, chỗ ngồi -> NONE. Mỗi aspect độc lập. NEGATIVE khi thiếu tài liệu, chật, ồn; POSITIVE khi phong phú, yên tĩnh; không rõ -> NEUTRAL."
        ),
        "giu_xe_ve_sinh": (
            "ĐÁNH GIÁ GIỮ XE/NHÀ VỆ SINH. Nếu câu không nói gửi xe hoặc nhà vệ sinh -> NONE. Mỗi aspect độc lập. NEGATIVE khi bẩn, đắt, mùi khó chịu; POSITIVE khi sạch, thuận tiện; không rõ -> NEUTRAL."
        ),
        "cong_quan_ly_dao_tao": (
            "ĐÁNH GIÁ CỔNG/TRANG QUẢN LÝ ĐÀO TẠO. Nếu câu không nhắc cổng đào tạo, đăng nhập, tra cứu -> NONE. Mỗi aspect độc lập. NEGATIVE khi quá tải, treo, khó dùng; POSITIVE khi ổn định, dễ dùng; không rõ -> NEUTRAL."
        ),
    },
    "khac": {
        "_default": (
            "ĐÁNH GIÁ NHÓM KHÁC (học phí, học bổng, hành chính, CLB, KTX, một cửa, đăng ký tín chỉ, điểm rèn luyện). Nếu câu không nhắc rõ đến nhóm này -> NONE. Mỗi aspect đánh giá độc lập (ví dụ: học phí tăng nhưng phòng học tốt -> nhóm khác NEGATIVE, cơ sở vật chất POSITIVE). NEGATIVE khi phàn nàn khó khăn, chậm trễ; POSITIVE khi khen rõ ràng, nhanh chóng; không rõ -> NEUTRAL."
        ),
        "hoc_phi": (
            "ĐÁNH GIÁ HỌC PHÍ. Nếu câu không nhắc học phí, mức thu, đóng tiền -> NONE. Mỗi aspect độc lập. NEGATIVE khi đắt, tăng, thiếu minh bạch; POSITIVE khi hợp lý, minh bạch; không rõ -> NEUTRAL."
        ),
        "hoc_bong": (
            "ĐÁNH GIÁ HỌC BỔNG. Nếu câu không nói tiêu chí, quy trình, kết quả học bổng -> NONE. Mỗi aspect độc lập. NEGATIVE khi khó, chậm, không rõ; POSITIVE khi dễ, minh bạch, kịp thời; không rõ -> NEUTRAL."
        ),
        "hanh_chinh": (
            "ĐÁNH GIÁ THỦ TỤC HÀNH CHÍNH/CTSV. Nếu câu không nhắc hồ sơ, giấy tờ, xử lý -> NONE. Mỗi aspect độc lập. NEGATIVE khi rườm rà, chậm, thiếu phản hồi; POSITIVE khi nhanh, rõ ràng; không rõ -> NEUTRAL."
        ),
        "clb": (
            "ĐÁNH GIÁ CLB/HOẠT ĐỘNG NGOẠI KHÓA. Nếu câu không nói CLB, sự kiện, hoạt động sinh viên -> NONE. Mỗi aspect độc lập. NEGATIVE khi ít hoạt động, thiếu hấp dẫn; POSITIVE khi sôi nổi, hữu ích; không rõ -> NEUTRAL."
        ),
        "ktx": (
            "ĐÁNH GIÁ KÝ TÚC XÁ. Nếu câu không nhắc phòng KTX, an ninh, điện nước -> NONE. Mỗi aspect độc lập. NEGATIVE khi chật, mất an ninh, thiếu điện nước; POSITIVE khi sạch, an toàn, đầy đủ; không rõ -> NEUTRAL."
        ),
        "mot_cua": (
            "ĐÁNH GIÁ VĂN PHÒNG MỘT CỬA. Nếu câu không nhắc một cửa, tiếp nhận, trả kết quả -> NONE. Mỗi aspect độc lập. NEGATIVE khi chờ lâu, đông, xử lý chậm; POSITIVE khi nhanh, rõ ràng; không rõ -> NEUTRAL."
        ),
        "dang_ky_tin": (
            "ĐÁNH GIÁ ĐĂNG KÝ TÍN CHỈ. Nếu câu không nói đăng ký môn, hệ thống đăng ký -> NONE. Mỗi aspect độc lập. NEGATIVE khi quá tải, lỗi, khó dùng; POSITIVE khi ổn định, dễ dùng; không rõ -> NEUTRAL."
        ),
        "diem_ren_luyen": (
            "ĐÁNH GIÁ ĐIỂM RÈN LUYỆN. Nếu câu không nhắc DRL, minh chứng, quy trình -> NONE. Mỗi aspect độc lập. NEGATIVE khi khó, không công bằng; POSITIVE khi rõ ràng, công bằng; không rõ -> NEUTRAL."
        ),
    },
}

SUBTOPIC_KW = {
    "giang_vien": {
        "dung_gio": _no_diacritics_set({
            "đi dạy","lên lớp","vào lớp","bắt đầu tiết","kết thúc tiết",
            "giảng viên","giáo viên","thầy giáo","cô giáo","thầy cô",
            "giảng viên đi dạy","giảng viên lên lớp","giảng viên vào lớp",
            "thầy đi dạy","cô đi dạy","thầy lên lớp","cô lên lớp"
        }),
        "cham_diem": _no_diacritics_set({
            "chấm điểm","thang điểm","điểm thi","điểm thành phần","điểm tổng kết","phúc khảo",
            "điểm giữa kỳ","điểm cuối kỳ","điểm nhóm","điểm cá nhân","điểm bonus",
            "điểm chuyên cần","điểm chuyên đề","rubric","grading",
            "giảng viên","giáo viên","thầy giáo","cô giáo",
            "giảng viên chấm điểm","thầy chấm điểm","cô chấm điểm","giáo viên chấm điểm",
            "thầy giáo chấm điểm","cô giáo chấm điểm"
        }),
        "ho_tro": _no_diacritics_set({
            "tư vấn học tập","giải đáp học tập","phản hồi học tập","cvht",
            "cố vấn học tập","hướng dẫn học tập","trao đổi học tập","hỏi đáp học tập",
            "tư vấn sinh viên","giải đáp sinh viên","phản hồi sinh viên",
            "cố vấn sinh viên","hướng dẫn sinh viên",
            "giảng viên","giáo viên","thầy giáo","cô giáo",
            "giảng viên tư vấn","giảng viên hướng dẫn","giảng viên giải đáp",
            "thầy tư vấn","cô tư vấn","thầy hướng dẫn","cô hướng dẫn",
            "thầy giáo tư vấn","cô giáo tư vấn"
        }),
        "thai_do": _no_diacritics_set({
            "thái độ","ứng xử","tác phong","phong thái","giao tiếp","cách nói",
            "ngữ điệu","hành vi","cử chỉ","cách cư xử","thái độ giảng viên",
            "phong cách","tương tác","thái độ lớp","ngôn ngữ cơ thể",
            "giảng viên","giáo viên","thầy giáo","cô giáo",
            "thái độ thầy","thái độ cô","thái độ giáo viên",
            "thầy giáo thái độ","cô giáo thái độ","giảng viên thái độ"
        }),
        "giang_day": _no_diacritics_set({
            "giảng dạy","truyền đạt","diễn đạt","ví dụ","bài giảng","slide","ghi chú",
            "ôn tập","bài học","phương pháp","thực hành","lý thuyết",
            "thảo luận","minh họa","slide giảng","slide bài","giải thích","phong cách giảng",
            "giảng viên","giáo viên","thầy giáo","cô giáo",
            "giảng viên giảng dạy","thầy giảng","cô giảng","giáo viên giảng dạy",
            "thầy giáo giảng dạy","cô giáo giảng dạy"
        }),
    },

    "chuong_trinh": {
        "lich_hoc": _no_diacritics_set({
            "lịch học","thời khóa biểu","thời khoá biểu","kế hoạch học tập","xếp lịch","trùng lịch",
            "đổi lịch","lịch thi","lịch học thêm","ca tối","online","offline","ca sáng",
            "ca chiều","học bù","thi dồn","thi liên tục","xếp ca","thời gian học","lịch kiểm tra"
        }),
        "tin_chi": _no_diacritics_set({
            "tín chỉ","học phần","tiên quyết","song hành","đăng ký học phần","nợ môn",
            "đủ tín","số tín","khối lượng học","điều kiện học phần","mã môn","tải học",
            "phân bổ học phần","lộ trình học","số học phần"
        }),
        "de_cuong": _no_diacritics_set({
            "đề cương","syllabus","giáo trình","tài liệu bắt buộc môn học","tài liệu tham khảo môn học",
            "mục tiêu học phần","kế hoạch môn","outline","kế hoạch giảng dạy","phân bổ điểm",
            "tài liệu học môn học","hướng dẫn môn học","phân phối chương trình","khung điểm môn học","thang đánh giá môn học"
        }),
        "noi_dung": _no_diacritics_set({
            "nội dung","thực tế","thực tiễn","lộ trình","khung chương trình",
            "cập nhật","định hướng nghề","kiến thức","module",
            "chuyên đề","cấu trúc môn","chương trình học","đề mục","môn học","học liệu"
        }),
    },

    "co_so_vat_chat": {
        "mang": _no_diacritics_set({
            "mạng wifi","wifi","wi-fi","wi fi","đăng nhập wifi", "ping wifi","băng thông wifi","wifi trường"
        }),
        "phong_hoc": _no_diacritics_set({
            "phòng học","ánh sáng","đèn phòng học","máy lạnh","điều hòa","điều hoà","quạt",
            "bàn ghế phòng học","ổ điện phòng học","ổ cắm phòng học","cách âm","sàn nhà","rèm cửa","trần nhà","bảng viết"
        }),
        "phong_thi_nghiem": _no_diacritics_set({
            "phòng thí nghiệm","phòng thực hành","lab","phòng lab","máy thực hành",
            "cài phần mềm","thiết bị thí nghiệm","dụng cụ lab","phòng máy","thiết bị lab"
        }),
        "thiet_bi": _no_diacritics_set({
            "máy chiếu","micro","mic","loa","âm thanh","tivi","cáp","hdmi","adapter",
            "thiết bị giảng dạy","máy quay","camera lớp","loa bluetooth","âm lượng",
            "đầu nối","bộ chia","thiết bị phòng học","tv phòng học"
        }),
        "thu_vien": _no_diacritics_set({
            "thư viện","mượn sách","trả sách","tài liệu số thư viện","ebook thư viện","chỗ ngồi thư viện",
            "bàn đọc","yên tĩnh thư viện","giờ mở cửa thư viện","mượn giáo trình","tra cứu sách","wifi thư viện",
            "tra cứu thư viện","kệ sách","tài nguyên số thư viện","khu đọc","mượn tài liệu thư viện","mượn thiết bị thư viện",
            "tài liệu thư viện","tài liệu mượn thư viện","sách thư viện"
        }),
        "giu_xe_ve_sinh": _no_diacritics_set({
            "bãi giữ xe","nhà giữ xe","gửi xe","thẻ xe","quẹt thẻ","phí gửi xe",
            "nhà vệ sinh","toilet","giấy vệ sinh","nước rửa tay",
            "ống nước nhà vệ sinh","cống thoát nhà vệ sinh","sàn nhà vệ sinh","wc nhà vệ sinh"
        }),
        "cong_quan_ly_dao_tao": _no_diacritics_set({
            "trang quản lý đào tạo","cổng đào tạo","hệ thống đào tạo","portal","cổng thông tin",
            "đăng nhập cổng đào tạo","quên mật khẩu","reset mật khẩu","quá tải","treo","tra cứu điểm",
            "web đào tạo","cổng sinh viên","hệ thống online","trang web đào tạo"
        }),
    },

    "khac": {
        "hoc_phi": _no_diacritics_set({
            "học phí","thu thêm","biên lai","miễn giảm","chính sách học phí","công khai học phí",
            "đóng tiền","nộp học phí","thu tiền","hoá đơn học phí","chính sách","phiếu thu","biên nhận",
            "đóng học","nộp lệ phí","phí học","thanh toán học phí","biên lai học phí","phiếu thu học phí"
        }),
        "hoc_bong": _no_diacritics_set({
            "học bổng","học bổng kkht","tiêu chí học bổng","điểm chuẩn học bổng",
            "nộp hồ sơ học bổng","kết quả học bổng","trễ hạn học bổng","xét học bổng",
            "điều kiện học bổng","quỹ học bổng","thông báo học bổng","hồ sơ học bổng","điểm xét"
        }),
        "hanh_chinh": _no_diacritics_set({
            "thủ tục hành chính","hành chính","giấy tờ hành chính","đóng dấu","xác nhận sinh viên","giấy xác nhận",
            "phòng ctsv","tiếp nhận hồ sơ hành chính","trả kết quả hành chính","xin giấy tờ hành chính","nộp hồ sơ hành chính","biểu mẫu hành chính",
            "phòng đào tạo hành chính","chứng nhận hành chính","xác minh hành chính","giấy phép hành chính","bản sao hành chính","văn thư hành chính"
        }),
        "clb": _no_diacritics_set({
            "câu lạc bộ","clb","tuyển thành viên","hoạt động clb","ngoại khóa","sự kiện","workshop",
            "đăng ký clb","đoàn hội","event","team","cuộc thi","hoạt động sv",
            "hoạt động ngoại khoá","nhóm sinh viên","sự kiện trường","đăng ký tham gia"
        }),
        "ktx": _no_diacritics_set({
            "ký túc xá","kí túc xá","ktx","ở ghép","phòng ktx","bảo vệ ktx","giờ giới nghiêm",
            "điện ktx","nước ktx","khu ở ktx","an ninh ktx","phòng chung ktx",
            "toà ktx","khu vực ở ktx","quản lý ktx"
        }),
        "mot_cua": _no_diacritics_set({
            "văn phòng một cửa","vp1c","phòng một cửa","nộp hồ sơ một cửa","số thứ tự","lấy giấy một cửa","trả giấy một cửa","trả kết quả một cửa",
            "hồ sơ một cửa","giấy tờ một cửa","số lượt một cửa","quầy tiếp nhận một cửa","một cửa"
        }),
        "dang_ky_tin": _no_diacritics_set({
            "đăng ký môn","đăng ký tín chỉ","đk tín","đk môn","server đăng ký",
            "xếp lịch tự động","lọc trùng lịch","hệ thống đăng ký",
            "đăng ký online","chọn môn","mở lớp","đóng lớp","sắp lịch","hệ thống đăng ký tín chỉ"
        }),
        "diem_ren_luyen": _no_diacritics_set({
            "điểm rèn luyện","drl","đánh giá rèn luyện","minh chứng drl","chấm drl",
            "minh chứng","điểm rl","bảng drl","đánh giá cá nhân","đánh giá tập thể"
        }),
    },
}


def _aspect_has_kw(aspect_vi: str, s_norm: str) -> bool:
    """Kiểm tra aspect có keyword trong sentence không (chỉ keywords >= 3 ký tự)"""
    for kws in SUBTOPIC_KW.get(aspect_vi, {}).values():
        for kw in kws:
            kw_norm = _norm_match(kw)
            if len(kw_norm) >= 3 and kw_norm in s_norm:
                return True
    return False

def _pick_subprompt(aspect: str, sentence: str) -> str:
    s = _norm_match(str(sentence))
    for sub, kws in SUBTOPIC_KW.get(aspect, {}).items():
        for kw in kws:
            kw_norm = _norm_match(kw)
            if len(kw_norm) >= 3 and kw_norm in s:
                return ASPECT_PROMPTS[aspect].get(sub, ASPECT_PROMPTS[aspect]["_default"])
    return ASPECT_PROMPTS[aspect]["_default"]


In [4]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


# Load Data


In [5]:
from pathlib import Path
import pandas as pd

train_df = pd.read_csv("train_data.csv")
val_df   = pd.read_csv("train.csv")
train_df["sentence"] = train_df["sentence"].apply(_norm_store).astype(str)
val_df["sentence"]   = val_df["sentence"].apply(_norm_store).astype(str)

train_df = pd.concat([train_df, val_df], ignore_index=True)
train_df = train_df.drop_duplicates(subset=["sentence"]).reset_index(drop=True)

topic_map = {v: i for i, v in enumerate(sorted(train_df["topic"].dropna().unique()))}
sent_map  = {v: i for i, v in enumerate(sorted(train_df["sentiment"].dropna().unique()))}

train_df["topic_id"]     = train_df["topic"].map(topic_map).astype(int)
train_df["sentiment_id"] = train_df["sentiment"].map(sent_map).astype(int)

print(f"✅ Loaded {len(train_df)} samples")


✅ Loaded 20221 samples


# Build Pairs & Dataset


In [6]:
# ===== Build train pairs =====
def build_pairs_df(base_df, neg_per_sent=1, rng_seed=42, skip_hard_negatives=True):
    """
    Build training pairs from base dataframe.

    Args:
        base_df: DataFrame with columns [sentence, topic_id, sentiment_id]
        neg_per_sent: Number of negative samples per sentence
        rng_seed: Random seed
        skip_hard_negatives: If True, tạo negative pairs có điều kiện:
                            - Ưu tiên easy negatives (aspects không có keywords)
                            - Tạo một phần hard negatives (tỷ lệ thấp) để model học reject aspects có keywords nhưng không liên quan
                            - Điều này giúp model học cả reject và multi-topic detection
    """
    rng = random.Random(rng_seed)
    rows = []

    for _, r in base_df.iterrows():
        s = str(r["sentence"])
        t_id = int(r["topic_id"])
        y = int(r["sentiment_id"])
        t_key = ASPECTS[t_id]

        rows.append({"text": s, "aspect": t_key, "label": y + 1})
       
        others = [a for a in ASPECTS if a != t_key]

        s_norm = _norm_match(s)
        hard = [a for a in others if _aspect_has_kw(a, s_norm)]
        easy = [a for a in others if a not in hard]

        if skip_hard_negatives:
            picks = []

   
            if easy:
                n_easy = min(neg_per_sent, len(easy))
                picks.extend(rng.sample(easy, k=n_easy))

            if hard and len(picks) < neg_per_sent:
                n_hard = min(1, neg_per_sent - len(picks))
                picks.extend(rng.sample(hard, k=min(n_hard, len(hard))))
            for a in picks:
                rows.append({"text": s, "aspect": a, "label": 0})
        else:
            picks = []
            if hard:
                hard_shuf = hard[:]
                rng.shuffle(hard_shuf)
                picks.extend(hard_shuf[:neg_per_sent])
            if len(picks) < neg_per_sent and easy:
                rest = neg_per_sent - len(picks)
                picks.extend(rng.sample(easy, k=min(rest, len(easy))))

            for a in picks[:neg_per_sent]:
                rows.append({"text": s, "aspect": a, "label": 0})

    return pd.DataFrame(rows)


# ---- tokenizer ----
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=False)

def encode_pair(tokenizer, text, prompt, max_len):
    return tokenizer(
        prompt, text,
        truncation="only_second",
        padding="max_length",
        max_length=max_len,
        return_tensors="pt",
        return_overflowing_tokens=False
    )


# ---- dataset ----
class PairABSADataset(Dataset):
    def __init__(self, df_pairs, tokenizer, max_len=MAX_LEN):
        self.df = df_pairs.reset_index(drop=True)
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        s = self.df.loc[i, "text"]
        a = self.df.loc[i, "aspect"]

        prompt = _pick_subprompt(a, s)
        y = int(self.df.loc[i, "label"])
        enc = encode_pair(self.tok, s, prompt, self.max_len)

        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(y, dtype=torch.long)
        return item


# ---- dataloader ----
def make_loader(df_pairs, tokenizer, batch_size=64, max_len=MAX_LEN, shuffle=True):
    ds = PairABSADataset(df_pairs, tokenizer, max_len=max_len)
    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
    )


# Define Model & Optimizer


In [7]:
# === Model: Head 2 tầng + GELU + LayerNorm ===
class PhoBERTPairABSA(nn.Module):
    def __init__(self, base_model="vinai/phobert-base", num_cls=4, dropout=0.3):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(base_model)
        hidden_size = self.backbone.config.hidden_size
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.LayerNorm(hidden_size),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_cls)
        )

    def forward(self, input_ids, attention_mask):
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:, 0, :]
        return self.classifier(cls)

model = PhoBERTPairABSA(base_model=BASE_MODEL, num_cls=4, dropout=0.3).to(device)
optimizer = AdamW([
    {"params": model.backbone.parameters(), "lr": LR_BACKBONE},
    {"params": model.classifier.parameters(), "lr": LR_HEAD},
], weight_decay=WEIGHT_DECAY)
scaler = GradScaler(enabled=(device.type == "cuda"))
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=LR_BACKBONE/10)


# Train

In [8]:
from sklearn.metrics import f1_score, precision_recall_fscore_support
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight


model.train()
for epoch in range(1, NUM_EPOCHS + 1):
    # ====== Build TRAIN  ======
    neg_ps = 1 if epoch <= 4 else NEG_PER_SENT
    seed_ep = SEED + epoch

    train_pairs_df = build_pairs_df(
        base_df=train_df[["sentence","topic_id","sentiment_id"]],
        neg_per_sent=neg_ps,
        rng_seed=seed_ep,
        skip_hard_negatives=True 
    )
    # class weight
    labels_4 = train_pairs_df["label"].values
    counts = np.bincount(labels_4, minlength=4).astype(np.int64)
    w = compute_class_weight("balanced", classes=np.arange(4), y=labels_4).astype(np.float32)

    alpha = 0.18 if epoch <= 2 else (0.15 if epoch <= 4 else 0.12)
    w0_floor = np.max(w[1:]) * alpha
    w[0] = max(w[0], w0_floor)
    print(f"\n[Epoch {epoch}] label counts (none,neg,neu,pos) = {counts.tolist()} | "
          f"class_weight = {w.round(3).tolist()} | neg_per_sent={neg_ps}")

    criterion = nn.CrossEntropyLoss(
        weight=torch.tensor(w, dtype=torch.float32, device=device),
        label_smoothing=LABEL_SMOOTH
    )

    train_loader = make_loader(
        train_pairs_df,
        tokenizer,
        batch_size=BATCH_SIZE,
        max_len=MAX_LEN,
        shuffle=True
    )


    # ====== Train one epoch ======
    running_loss = 0.0
    y_true_all, y_pred_all = [], []

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS}",
                leave=False, dynamic_ncols=True, file=sys.stdout)

    for batch in pbar:
        input_ids      = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        labels         = batch["labels"].to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with autocast(device_type="cuda", dtype=torch.float16, enabled=(device.type == "cuda")):
            logits = model(input_ids, attention_mask)
            loss   = criterion(logits, labels)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP_GRAD_NORM)
        scaler.step(optimizer); scaler.update()

        running_loss += loss.item()
        preds = logits.argmax(dim=1)

        y_true_all.extend(labels.cpu().tolist())
        y_pred_all.extend(preds.cpu().tolist())

        pbar.set_postfix(loss=running_loss / max(1, pbar.n))

    print("👉 Label distribution SEEN:", Counter(y_true_all))

    # === Train metrics ===
    y_true_all = np.array(y_true_all)
    y_pred_all = np.array(y_pred_all)

    acc4 = (y_pred_all == y_true_all).mean()
    mask_pos = (y_true_all != 0)
    acc_pos3 = (y_pred_all[mask_pos] == y_true_all[mask_pos]).mean() if mask_pos.any() else 0.0
    macro_f1_pos3 = f1_score(y_true_all[mask_pos], y_pred_all[mask_pos],
                             average="macro", labels=[1,2,3]) if mask_pos.any() else 0.0

    y_true_none = (y_true_all == 0).astype(int)
    y_pred_none = (y_pred_all == 0).astype(int)
    none_prec, none_rec, none_f1, _ = precision_recall_fscore_support(
        y_true_none, y_pred_none, average="binary", zero_division=0
    )

    avg_loss = running_loss / len(train_loader)

    print(
        f"\n✅ Epoch {epoch}/{NUM_EPOCHS} | "
        f"TrainLoss={avg_loss:.4f} | Acc4={acc4:.4f} | "
        f"Acc_pos3={acc_pos3:.4f} | MacroF1_pos3={macro_f1_pos3:.4f} | "
        f"None(P/R/F1)={none_prec:.3f}/{none_rec:.3f}/{none_f1:.3f}"
    )

    scheduler.step()

print("\n🎉 Training completed.")



[Epoch 1] label counts (none,neg,neu,pos) = [20221, 9045, 1501, 9675] | class_weight = [1.2120000123977661, 1.1180000305175781, 6.736000061035156, 1.0449999570846558] | neg_per_sent=1


Epoch 1/10:   0%|          | 0/632 [00:00<?, ?it/s]

👉 Label distribution SEEN: Counter({0: 20221, 3: 9675, 1: 9045, 2: 1501})

✅ Epoch 1/10 | TrainLoss=1.0096 | Acc4=0.7619 | Acc_pos3=0.6430 | MacroF1_pos3=0.6318 | None(P/R/F1)=0.825/0.881/0.852

[Epoch 2] label counts (none,neg,neu,pos) = [20221, 9045, 1501, 9675] | class_weight = [1.2120000123977661, 1.1180000305175781, 6.736000061035156, 1.0449999570846558] | neg_per_sent=1


Epoch 2/10:   0%|          | 0/632 [00:00<?, ?it/s]

👉 Label distribution SEEN: Counter({0: 20221, 3: 9675, 1: 9045, 2: 1501})

✅ Epoch 2/10 | TrainLoss=0.8186 | Acc4=0.8717 | Acc_pos3=0.8193 | MacroF1_pos3=0.8095 | None(P/R/F1)=0.894/0.924/0.909

[Epoch 3] label counts (none,neg,neu,pos) = [20221, 9045, 1501, 9675] | class_weight = [1.0099999904632568, 1.1180000305175781, 6.736000061035156, 1.0449999570846558] | neg_per_sent=1


...

👉 Label distribution SEEN: Counter({0: 40437, 3: 9675, 1: 9045, 2: 1501})

✅ Epoch 7/10 | TrainLoss=0.7925 | Acc4=0.9465 | Acc_pos3=0.9184 | MacroF1_pos3=0.9236 | None(P/R/F1)=0.972/0.961/0.966

[Epoch 8] label counts (none,neg,neu,pos) = [40437, 9045, 1501, 9675] | class_weight = [1.2120000123977661, 1.6770000457763672, 10.102999687194824, 1.5670000314712524] | neg_per_sent=2


Epoch 8/10:   0%|          | 0/948 [00:00<?, ?it/s]

👉 Label distribution SEEN: Counter({0: 40437, 3: 9675, 1: 9045, 2: 1501})

✅ Epoch 8/10 | TrainLoss=0.7849 | Acc4=0.9491 | Acc_pos3=0.9232 | MacroF1_pos3=0.9288 | None(P/R/F1)=0.974/0.962/0.968

[Epoch 9] label counts (none,neg,neu,pos) = [40437, 9045, 1501, 9675] | class_weight = [1.2120000123977661, 1.6770000457763672, 10.102999687194824, 1.5670000314712524] | neg_per_sent=2


Epoch 9/10:   0%|          | 0/948 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Export Model


In [9]:
#Export
import os, torch

SAVE_DIR = "/content/phobert_pair_absa"
os.makedirs(SAVE_DIR, exist_ok=True)

model_bin_path = os.path.join(SAVE_DIR, "model.bin")
torch.save(model.state_dict(), model_bin_path)
print(f"✅ Saved model weights to: {model_bin_path}")

tok_dir = os.path.join(SAVE_DIR, "tokenizer")
os.makedirs(tok_dir, exist_ok=True)
tokenizer.save_pretrained(tok_dir)
print(f"✅ Saved tokenizer to: {tok_dir}")


✅ Saved model weights to: /content/phobert_pair_absa/model.bin
✅ Saved tokenizer to: /content/phobert_pair_absa/tokenizer


In [10]:
from huggingface_hub import HfApi, login
import os

MODEL_REPO = "Ptul2x5/Student_Feedback_Sentiment"
HF_TOKEN = "my-secret-token"

login(token=HF_TOKEN)
api = HfApi()

print("📤 Uploading tokenizer...")
api.upload_folder(
    folder_path=os.path.join(SAVE_DIR, "tokenizer"),
    repo_id=MODEL_REPO,
    repo_type="model",
    commit_message="Upload tokenizer for inference"
)
print("✅ Tokenizer uploaded successfully")

print("📤 Uploading model.bin...")
api.upload_file(
    path_or_fileobj=os.path.join(SAVE_DIR, "model.bin"),
    path_in_repo="model.bin",
    repo_id=MODEL_REPO,
    repo_type="model",
    commit_message="Upload model weights (final model from training)"
)
print("✅ Model weights uploaded successfully")

📤 Uploading tokenizer...


No files have been modified since last commit. Skipping to prevent empty commit.


✅ Tokenizer uploaded successfully
📤 Uploading model.bin...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...obert_pair_absa/model.bin:   0%|          |  733kB /  542MB            

✅ Model weights uploaded successfully
